In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('data/skylab_instagram_datathon_dataset.csv', delimiter=';')

#drop column period, primary exchange name and calculation type, 
df = df.drop(['period', 'primary_exchange_name', 'calculation_type', 'domicile_country_name'], axis=1)

#sort by period end date
df = df.sort_values(by='period_end_date', ascending=True)

#df = df[df['ultimate_parent_legal_entity_name'] == 'SWATCH GROUP AG']

#just show rows with  name hering
df = df[df['period_end_date'] > '2020-12-31']




#just take rows with period end date higher than 2020-12-31
#df = df[df['period_end_date'] > '2020-12-31']

display(df.head(5))


,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,ultimate_parent_legal_entity_name,followers,pictures,videos,comments,likes
195569,2021-01-02,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,Zales,Signet Jewelers Ltd,Signet Jewelers Ltd,183163.0,32.0,7.0,505.0,36174.0
687138,2021-01-02,Beauty & Boutique,Beauty & Boutique,Est,KAO,KAO,16503.0,1.0,0.0,1.0,310.0
167814,2021-01-02,Luxury & Premium & Mainstream,Soft Luxury,Valentino,Valentino,Mayhoola for Investments Spc,14846655.0,87.0,19.0,5271.0,795776.0
153520,2021-01-02,Apparel Retail,Ecommerce Mainstream Apparel Retail,Wish,ContextLogic Inc,ContextLogic Inc,7277002.0,126.0,41.0,3162.0,122204.0
56891,2021-01-02,Apparel Retail,US Softlines Analyst Interest List,Banana Republic,Gap,Gap,2040463.0,182.0,14.0,4631.0,314958.0


In [2]:
columns = df.columns
print(columns)
df_sum = 0

for i in columns:
    
    print("\n")
    print("-----------------------------------------------------------------------------")
    print(i, "--> has length:", len(df[i].unique()))
    df_i = df[i].value_counts().reset_index()
    df_i.columns = [i, "count"]
    display(df_i)
    df_sum = df_i["count"].sum()
    print("Sum of count:", df_sum)
    df_sum = 0
    print("-----------------------------------------------------------------------------")
    print("\n")

#df_period = df["period"].value_counts().reset_index()
#df_period.columns = ["period", "count"]
#df_period


Index(['period_end_date', 'compset_group', 'compset',
       'business_entity_doing_business_as_name', 'legal_entity_name',
       'ultimate_parent_legal_entity_name', 'followers', 'pictures', 'videos',
       'comments', 'likes'],
      dtype='object')


-----------------------------------------------------------------------------
period_end_date --> has length: 142


,period_end_date,count
0,2023-09-16,1635
1,2023-06-17,1635
2,2023-03-25,1635
3,2023-04-01,1635
4,2023-04-08,1635
...,...,...
137,2021-03-06,1628
138,2021-03-13,1628
139,2021-03-20,1628
140,2021-03-27,1628


Sum of count: 231633
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
compset_group --> has length: 20


,compset_group,count
0,Luxury & Premium & Mainstream,69296
1,Apparel Retail,68780
2,Sportswear & Athleisure,18176
3,Beauty & Boutique,14768
4,Beverages,14626
5,Food Products,14200
6,Restaurants,11786
7,Dermatology and Orthodontics,3524
8,Outdoor Gear,3124
9,Fitness & Exercise,2982


Sum of count: 231633
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
compset --> has length: 54


...

Sum of count: 231510
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
comments --> has length: 26383


,comments,count
0,0.0,5816
1,10.0,290
2,42.0,276
3,56.0,274
4,3.0,273
...,...,...
26377,35594.0,1
26378,7922.0,1
26379,8484.0,1
26380,146492.0,1


Sum of count: 231121
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
likes --> has length: 77097


,likes,count
0,0.0,5130
1,191.0,40
2,266.0,35
3,233.0,33
4,79.0,28
...,...,...
77091,10307.0,1
77092,9745.0,1
77093,276486.0,1
77094,70780903.0,1


Sum of count: 231537
-----------------------------------------------------------------------------




In [3]:
display(df.describe())

,followers,pictures,videos,comments,likes
count,2.293190e+05,231491.000000,231510.000000,2.311210e+05,2.315370e+05
mean,1.675495e+07,316.704891,84.377578,3.264462e+04,1.450647e+06
std,1.379594e+08,2612.833626,758.696448,2.940790e+05,1.200478e+07
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,3.731370e+05,15.000000,3.000000,3.190000e+02,1.191500e+04
50%,1.289569e+06,36.000000,9.000000,1.265000e+03,6.253500e+04
75%,5.440812e+06,104.000000,27.000000,5.201000e+03,2.881020e+05
max,3.502565e+09,135714.000000,35905.000000,1.605335e+07,4.583853e+08


In [4]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231633 entries, 195569 to 188398
Data columns (total 11 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   period_end_date                         231633 non-null  object 
 1   compset_group                           231633 non-null  object 
 2   compset                                 231633 non-null  object 
 3   business_entity_doing_business_as_name  231633 non-null  object 
 4   legal_entity_name                       222971 non-null  object 
 5   ultimate_parent_legal_entity_name       222971 non-null  object 
 6   followers                               229319 non-null  float64
 7   pictures                                231491 non-null  float64
 8   videos                                  231510 non-null  float64
 9   comments                                231121 non-null  float64
 10  likes                                  

None

In [5]:
df = df[df['ultimate_parent_legal_entity_name'] == 'KAO']
df.head(5)

,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,ultimate_parent_legal_entity_name,followers,pictures,videos,comments,likes
687138,2021-01-02,Beauty & Boutique,Beauty & Boutique,Est,KAO,KAO,16503.0,1.0,0.0,1.0,310.0
70289,2021-01-02,Beauty & Boutique,Beauty & Boutique,Sofina,KAO,KAO,23461.0,6.0,1.0,6.0,962.0
453750,2021-01-02,Beauty & Boutique,Beauty & Boutique,John Frieda,KAO,KAO,526108.0,219.0,38.0,9036.0,39475.0
325208,2021-01-02,Beauty & Boutique,Beauty & Boutique,Sensai,KAO,KAO,NaN,112.0,2.0,6694.0,22223.0
330813,2021-01-02,Beauty & Boutique,Beauty & Boutique,Kanebo,KAO,KAO,NaN,28.0,7.0,615.0,3800.0


In [6]:
# use seaborn to create a scatter plot of the data
#sns.pairplot(df)
#plt.show()

In [7]:
# use seaborn to create a scatter plot of the data
#sns.pairplot(df, hue='category')
#plt.show()

In [8]:
# use seaborn to create a heatmap of the data
#sns.heatmap(df.corr(), annot=True)
#plt.show()